<a href="https://colab.research.google.com/github/SilvioIA2025/ALura2025/blob/main/Projeto_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
%pip -q install google-genai

In [230]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [233]:
# Configura o cliente da SDK do Gemini

from google import genai
client = genai.Client()
from IPython.display import HTML, Markdown



In [234]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [235]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [262]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
import textwrap
from rich.markdown import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [263]:
##################################################################
# --- Agente 1: Buscador de Custos de Viagens Internacionais --- #
##################################################################
def agente_buscador(topico, pais, perfil, data_de_inicio, data_de_fim, numero_adultos, numero_crianças):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em calcular gastos em viagens internacionais, meu objetivo é fornecer uma estimativa detalhada e otimizada dos seus custos,
        considerando suas preferências e buscando as melhores opções em cada categoria.

        Instruções para o Especialista em Cálculos de Viagem:

        1. Compreensão do Perfil do Viajante:

        Destino: País(es) e Cidade(s).
        Período: Datas de partida e retorno (para considerar alta/baixa temporada e eventos).
        Número de Viajantes: Adultos e crianças.
        Estilo de Viagem: (e.g., mochileiro, econômico, confortável, luxuoso) para calibrar os valores.
        Interesses Específicos: (e.g., gastronomia, museus, aventura, compras) para alocar orçamento.

        2. Análise e Estimativa por Categoria:

        Voos (Melhores Opções):
        Pesquisar e comparar companhias aéreas, escalas, duração do voo e aeroportos de partida/chegada.
        Considerar voos diretos vs. com escalas (custo/benefício).
        Avaliar épocas de compra (antecedência) e flexibilidade de datas para melhores preços.
        Incluir custos de bagagem despachada, se aplicável.

        Aluguel de Carro:
        Avaliar a necessidade do aluguel versus transporte público no destino.
        Selecionar o tipo de veículo adequado (econômico, SUV, etc.).
        Incluir custos de seguro obrigatório e opcionais, combustível, pedágios e estacionamento.
        Comparar locadoras e políticas de retirada/devolução.

        Hotéis/Hospedagem (Melhores Opções):
        Sugerir tipos de hospedagem (hotéis, hostels, Airbnb, pousadas) com base no estilo de viagem.
        Considerar localização (proximidade a atrações, transporte), avaliações, comodidades (café da manhã, Wi-Fi) e política de cancelamento.
        Estimar custo por noite, ajustado pela temporada.

        Refeições:
        Estimar um custo médio diário por pessoa para café da manhã, almoço e jantar, diferenciando entre restaurantes econômicos, médios e sofisticados.
        Considerar o custo de lanches, bebidas e gorjetas.
        Perguntar se o viajante pretende cozinhar algumas refeições.

        Atrações Turísticas e Atividades:
        Listar as principais atrações pagas e o custo de seus ingressos.
        Sugerir passes turísticos (se vantajosos para a quantidade de atrações).
        Incluir custo de tours guiados, shows ou atividades específicas desejadas.

        Transporte Local:
        Custos de metrô, ônibus, bondes, táxis ou aplicativos de transporte (Uber, Lyft, etc.).
        Estimar passes diários/semanais se forem mais econômicos.
        Despesas Gerais e Extras:

        Seguro Viagem: Item obrigatório e essencial.
        Visto e Taxas de Entrada: Se aplicável ao destino.
        Comunicações: Custo de chip local ou roaming internacional.
        Emergências/Imprevistos: Sugerir uma reserva de 10-15% do total para gastos não planejados.
        Compras/Souvenirs: Definir um orçamento para compras pessoais e lembrancinhas.
        Outros: Lavanderia, gorjetas adicionais, pequenos gastos diversos.

        3. Saída Final:
        Faça voce a escolha de uma cidade a ser visitada no pais, caso o não tenha sido escolhida, retorne todos os resultados de custo pelo
        estilo de Viagem selecionado no perfil (mochileiro, economico, confortavel e luxuoso) e voce deve escolher automaticamente dois
        Interesses Específicos: dentre Gastronomia, Museus, Aventura e Compras.
        Apresentar um orçamento detalhado por categoria (Voos, Aluguel de Carro, Hotéis/Hospedagem, Refeições, Atrações Turísticas e Atividades, Transporte Local, Seguro Viagem
        Visto e Taxas de Entrada, Comunicações, Emergências/Imprevistos, Compras/Souvenirs e Outros ) , com uma estimativa total convertida para a moeda de origem do viajante
        (geralmente BRL) e da moeda local do destino, utilizando a taxa de câmbio atualizada. Oferecer dicas de economia e personalização.

        A sua tarefa é usar a ferramenta de busca do google (google_search) respeitando a data inicial e final da viagem
        Foque em no maximo 4 opções relevantes respeitando os estilos de Viagem: (mochileiro, econômico, confortável, luxuoso), com base no preço, categoria, e avaliaçoes dos clientes anteriores.
        Se uma avaliação tiver poucas informações ou reações entusiasmadas, é possivel que ela não seja tão relevante assim
        e pode ser substituido por outro que tenha mais.
        Essas avaliações relevantes devem ser atuais, de no maximo um tres meses da data de hoje.
        """,
        description = "Agente que busca melhores voos, carros, hoteis, refeiçoes, atrações turisticas, transporte e custos diretos e indiretos de viagem indicado no Google",
        tools=[google_search],
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nPaís: {pais}\nPerfil: {perfil}\nData de inicio: {data_de_inicio}\nData de fim: {data_de_fim}\nNumero_adultos: {numero_adultos}\nNumero_crianças: {numero_crianças}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [264]:
##########################################
# --- Agente 2: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, lancamentos_buscados):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em documentos profissionais.
            Por ter um público amplo, entre 18 e 70 anos, use sempre um tom de escrita adequado.
            Revise o lancamentos_buscados abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Separar por uma linha em branco cada item da lista.
            Se o lancamentos_buscados estiver bom, repita apenas o texto novamente'.
            Caso haja problemas, corrija e refaça o texto de maneira adequada, gerando uma versão corrigida
            """,
        description="Agente revisor de post para documentos profissionais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {lancamentos_buscados}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [265]:
##################################################################
# --- Agente 3: Buscador da Melhor alternativa de levar ME --- #
##################################################################
def agente_moeda(topico, pais):
    imagem = Agent(
        name="agente_moeda",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Voce como especialista em cambio da Viktap, por favor me faça um breve resumo de qual a melhor maneira para eu converter o meu
        dinheiro local para o dinheiro internacional, considerando cartao de credito, cartao de debito, cartao internacional, remessa para conta
        no estrangeiro, ou PIX. No caso do PIX, fale sobre o viktap consultando a fonte no site https://viktap.com.br/
        """,
        tools=[google_search],
    )
    entrada_do_agente_moeda = f"Tópico: {topico}\nPais: {pais}"
    # Executa o agente
    escolha_moeda = call_agent(imagem, entrada_do_agente_moeda)
    return escolha_moeda

In [266]:
####################################################################
# --- Agente 4: Sugestao de Voo, pelo perfil escolhido ----------- #
####################################################################
def agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno):
    viagem = Agent(
        name="agente_viagem",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais da LATAM, faça apenas uma opção resumida para um voo consultando a LATAM saindo
        do campo aeroporto e na data de ida data_inicio e data de retorno data_retorno
        por favor escolha um voo direto de ida e um voo direto de volta a sua escolha informando o numero do voo e horario de partida e chegada
        """,
        tools=[google_search],
    )
    entrada_do_agente_viagem = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nAeroporto: {aeroporto}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    viagem_pais = call_agent(viagem, entrada_do_agente_viagem)
    return viagem_pais

In [267]:
################################################################
# --- Agente 5: Sugestao de Hotel, refeição e passeio turistico
#     pelo perfil escolhido ---------------------------------- #
################################################################
def agente_outros(topico, pais, perfil, data_inicio, data_retorno):
    outros = Agent(
        name="agente_outros",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais, faça apenas uma opção resumida para hoteis, refeição e passeios turisticos
        levando em consideração na data de ida data_inicio e data de retorno data_retorno
        por favor escolha somente 3 opções de hoteis, passeios turisticos e refeicoes.
        """,
        tools=[google_search],
    )
    entrada_do_agente_outros = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    outros = call_agent(outros, entrada_do_agente_outros)
    return outros

In [268]:
##agente_buscador(topico, pais, data_inicio, data_retorno, numero_adultos, numero_crianças)
##agente_revisor(topico, lancamentos_buscados)
##agente_moeda(topico, pais)
##agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
##agente_outros(topico, pais, perfil, data_inicio, data_retorno)

from datetime import datetime

def obter_data_do_usuario(param1):
    while True:
        if param1 == 'ida':
          data_str = input("Digite a data de inicio (DD/MM/AAAA): ")
        else:
          data_str = input("Digite a data de retorno (DD/MM/AAAA): ")

        try:
            # Tenta converter a string para um objeto datetime
            data_obj = datetime.strptime(data_str, "%d/%m/%Y")
            return data_obj
        except ValueError:
            print("Formato de data inválido. Por favor, use DD/MM/AAAA.")

def selecionar_perfil(opcoes, mensagem="Selecione uma opção:"):
    print(mensagem)
    for i, opcao in enumerate(opcoes):
        print(f"[{i + 1}] {opcao}")

    while True:
        try:
            escolha = input("Digite o número da sua escolha: ")
            indice_escolhido = int(escolha) - 1 # Ajusta para o índice da lista (0-based)

            if 0 <= indice_escolhido < len(opcoes):
                return opcoes[indice_escolhido]
            else:
                print("Número inválido. Por favor, digite um número da lista.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")



In [280]:
topico = 'Viagem'
pais = input("Digite o Pais/Cidade do destino: ")
data_inicio = obter_data_do_usuario('ida')
data_retorno = obter_data_do_usuario('fim')
adultos = input("Digite o numero de adultos: ")
criancas = input("Digite o numero de crianças: ")
opcoes = ["Mochileiro", "Economico", "Confortavel", "Luxuoso"]
perfil = selecionar_perfil(opcoes, mensagem="Selecione uma opção:")
print (perfil)
aeroporto = input("Digite o aeroporto de partida: ")

lancamentos_buscados = agente_buscador(topico, pais, perfil, data_inicio, data_retorno, adultos, criancas)
##print("\n Resultado do Agente 1 (Buscador)! \n")
##display(to_markdown(lancamentos_buscados))
##print("----------------------------------------------------------------")

post_final = agente_revisor(topico, lancamentos_buscados)
display(to_markdown("-------------------------------"))
display(to_markdown("Recomendações para a sua viagem"))
display(to_markdown("-------------------------------"))
display(to_markdown(post_final))

escolha_moeda = agente_moeda(topico, pais)
display(to_markdown("--------------------------------------------------"))
display(to_markdown("Dicas financeiras para pagar com moeda estrangeira"))
display(to_markdown("--------------------------------------------------"))
display(to_markdown(escolha_moeda))

viagem_pais = agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
display(to_markdown("------------------------------------"))
display(to_markdown("Dica de um agente de viagem da LATAM"))
display(to_markdown("------------------------------------"))
display(to_markdown(viagem_pais))

outros = agente_outros(topico, pais, perfil, data_inicio, data_retorno)
display(to_markdown("---------------------------------------------------------"))
display(to_markdown("Algumas dicas de Hoteis, refeiçoes e passeios turisticos"))
display(to_markdown("---------------------------------------------------------"))
display(to_markdown(outros))


Digite o Pais/Cidade do destino: Florença Italia
Digite a data de inicio (DD/MM/AAAA): 15/06/2025
Digite a data de retorno (DD/MM/AAAA): 30/06/2025
Digite o numero de adultos: 1
Digite o numero de crianças: 0
Selecione uma opção:
[1] Mochileiro
[2] Economico
[3] Confortavel
[4] Luxuoso
Digite o número da sua escolha: 3
Confortavel
Digite o aeroporto de partida: GRU


▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Recomendações para a sua viagem                                                                                

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Prezado(a) Cliente,                                                                                            
▌ Como seu especialista em viagens internacionais, preparei uma estimativa detalhada dos gastos para sua jornada 
▌ em Florença, Itália, de 15 a 30 de junho de 2025. Este planejamento considera seu perfil "Confortável" e foca  
▌ em seus interesses principais: "Gastronomia" e "Museus".                                                       
▌ É importante notar que junho é considerado alta temporada em Florença, o que pode influenciar os preços de voos
▌ e hospedagem. A taxa de câmbio utilizada para esta estimativa é 1 EUR = 6.32212 BRL (referência de 17 de maio  
▌ de 2025), mas lembre-se que ela pode variar até a data da sua viagem.                                          
▌ Estimativa de Gastos para 1 Pessoa (15/06/2025 a 30/06/2025 - 15 noites):                                      
▌ 1. Voos (ida e volta - São Paulo/Rio de Janeiro para Florença):                                                
▌ Melhores Opções Confortáveis: Para um estilo confortável, buscaremos voos que minimizem escalas e ofereçam um  
▌ bom nível de serviço. Companhias aéreas como ITA Airways, TAP Air Portugal e Lufthansa podem ser boas opções,  
▌ embora voos diretos para Florença a partir do Brasil sejam raros. É provável que haja pelo menos uma escala em 
▌ outra cidade europeia.                                                                                         
▌ Estimativa: Considerando a alta temporada e o estilo confortável, os voos podem variar significativamente.     
▌ Pesquisas indicam preços para junho de 2025 a partir de aproximadamente R$ 3.946 com a TAP Air Portugal. Para  
▌ voos com 1 parada para cidades próximas como Milão ou Roma em junho, o Skyscanner mostra opções a partir de    
▌ aproximadamente $599 (R$ 3.788). Voos com destino final em Florença tendem a ter um custo um pouco maior.      
▌ Nosso cálculo (estimativa confortável): EUR 950 - EUR 1.425 (R$ 6.000 - R$ 9.000) - Considerando voos com 1    
▌ escala e buscando conforto.                                                                                    
▌ 2. Aluguel de Carro:                                                                                           
▌ Em Florença, o centro histórico é compacto e muitas áreas são ZTL (Zona de Tráfego Limitado), restringindo o   
▌ acesso de carros. O transporte público é eficiente. Para um estilo confortável focado em gastronomia e museus  
▌ no centro, alugar um carro dentro da cidade pode ser desnecessário e caro (custos de estacionamento, taxas de  
▌ ZTL).                                                                                                          
▌ Nossa Sugestão: Não alugar carro na cidade. Utilize transporte público ou táxis/aplicativos conforme           
▌ necessidade.                                                                                                   
▌ Estimativa: EUR 0 (R$ 0)                                                                                       
▌ 3. Hotéis/Hospedagem (Estilo Confortável):                                                                     
▌ Tipos de Hospedagem: Hotéis boutique, hotéis com boas avaliações e localização central, com comodidades como   
▌ café da manhã e Wi-Fi incluídos.                                                                               
▌ Melhores Opções (com base em avaliações recentes e estilo confortável):                                        
▌  • Hotel Spadai: Moderno, boutique, próximo ao Duomo, quartos confortáveis. Preço estimado: $250-$400 por noite
▌    (aprox. R$ 1.580 - R$ 2.530).                                                                               
▌  • Palazzo Castri 1874: Mais relaxado, com piscina e spa, em área tranquila do centro. Preço estimado: $300-400
▌    por noite (aprox. R$ 1.895 - R$ 2.530).                                                                     
▌  • Hotel

...

▌ Como especialista em viagens internacionais da LATAM, apresento uma opção resumida de voo direto de ida e volta
▌ entre São Paulo (GRU) e a Itália, considerando o seu interesse em Florença. Embora Florença não possua voos    
▌ diretos do Brasil, as cidades de Roma e Milão são as portas de entrada principais na Itália com voos diretos   
▌ operados pela LATAM e outras companhias, sendo conveniente seguir de trem para Florença a partir delas.        
▌ Considerando a sua data de ida em 15/06/2025 e retorno em 30/06/2025, e buscando voos diretos, a LATAM opera   
▌ rotas para Roma (FCO) e Milão (MXP) partindo de São Paulo (GRU). A partir de junho de 2025, a LATAM aumentará a
▌ frequência de voos para a Itália.                                                                              
▌ Uma opção de voo direto para a sua viagem, buscando uma experiência confortável, seria para Milão (MXP), visto 
▌ que a LATAM terá voos diários para este destino a partir de junho de 2025.                                     
▌ Sugestão de voo LATAM (Sujeito a disponibilidade e alteração):                                                 
▌  • Voo de Ida:                                                                                                 
▌     • Número do Voo: LA8072 (Este voo opera em maio de 2025 com partida às 18:00 e chegada às 10:15 do dia     
▌       seguinte. É provável que um voo com horário similar esteja disponível em junho de 2025 com o aumento de  
▌       frequências mencionado.)                                                                                 
▌     • Origem: Aeroporto de Guarulhos (GRU), São Paulo                                                          
▌     • Destino: Aeroporto de Malpensa (MXP), Milão                                                              
▌     • Data de Partida: 15/06/2025                                                                              
▌     • Horário de Partida: Aproximadamente 18:00                                                                
▌     • Horário de Chegada: Aproximadamente 10:15 do dia 16/06/2025                                              
▌     • Duração aproximada: 11h 15m                                                                              
▌  • Voo de Volta:                                                                                               
▌     • Número do Voo: LA8073 (Este voo opera em maio de 2025 com partida às 13:00 e chegada às 19:45 do mesmo   
▌       dia. É provável que um voo com horário similar esteja disponível em junho de 2025.)                      
▌     • Origem: Aeroporto de Malpensa (MXP), Milão                                                               
▌     • Destino: Aeroporto de Guarulhos (GRU), São Paulo                                                         
▌     • Data de Retorno: 30/06/2025                                                                              
▌     • Horário de Partida: Aproximadamente 13:00                                                                
▌     • Horário de Chegada: Aproximadamente 19:45                                                                
▌     • Duração aproximada: 11h 45m                                                                              
▌ Para confirmar a disponibilidade e os horários exatos para as datas desejadas (15/06/2025 a 30/06/2025), é     
▌ fundamental realizar a consulta diretamente no site da LATAM ou com um agente de viagens. Os voos diretos da   
▌ LATAM entre GRU e MXP são operados em aeronaves Boeing 777-300ER ou Boeing 787-9.                              
▌ Ao chegar em Milão, você pode facilmente pegar um trem para Florença.                                          

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Algumas dicas de Hoteis, refeiçoes e passeios turisticos                                                       

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Para sua viagem confortável a Florença, Itália, entre 15 de junho de 2025 e 30 de junho de 2025, aqui estão 3  
▌ opções resumidas de hotéis, refeições e passeios turísticos:                                                   
▌ Hotéis (Perfil Confortável):                                                                                   
▌  1 Leone Blu Suites | UNA Esperienze: Localizado na Piazza Carlo Goldoni, este hotel é bem avaliado por seu    
▌    conforto.                                                                                                   
▌  2 Palazzo Vecchietti: Um palácio do século XVI transformado em hotel boutique de luxo no centro histórico,    
▌    oferecendo serviço personalizado e um ambiente elegante.                                                    
▌  3 Hotel La Gemma: Hotel 5 estrelas com quartos climatizados, Wi-Fi gratuito, estacionamento privativo,        
▌    restaurante, bar, sauna e banheira de hidromassagem.                                                        
▌ Refeições (Foco em Conforto e Qualidade):                                                                      
▌  1 Trattoria Cammillo: Um restaurante clássico e refinado, conhecido por pratos da culinária toscana e um      
▌    serviço atencioso em um ambiente confortável. Recomenda-se fazer reserva com semanas de antecedência.       
▌  2 Il Santo Bevitore: Um restaurante moderno em Florença com um ambiente aconchegante, pratos interessantes,   
▌    serviço consistente e uma excelente seleção de vinhos, atraindo moradores e visitantes.                     
▌  3 Osteria Cinghiale Bianco: Oferece uma experiência gastronômica mais rústica, mas acolhedora, com foco em    
▌    pratos fartos da culinária toscana, como o ensopado de javali.                                              
▌ Passeios Turísticos (Ritmo Confortável):                                                                       
▌  1 Explore o bairro Oltrarno: Conhecido por oficinas de artesãos, trattorias tradicionais e uma atmosfera mais 
▌    tranquila.                                                                                                  
▌  2 Visite os Jardins Bardini: Uma alternativa menos movimentada que os Jardins de Boboli, oferecendo vistas    
▌    deslumbrantes da cidade.                                                                                    
▌  3 Passeio tranquilo pela Piazza della Signoria e Ponte Vecchio ao pôr do sol: Caminhe pela praça, veja o      
▌    Palazzo Vecchio e a Loggia dei Lanzi, e depois siga em direção ao rio Arno para apreciar a vista do Ponte   
▌    Vecchio enquanto o sol se põe.                                                                              